In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
from tqdm import tqdm

In [2]:
# load resnet50 from torch which has been trained on imagenet
resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

# resnet50 = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

# save the pytorch model
torch.save(resnet50, './../TL_weightsv2/resnet50.pt')

Using cache found in /home/ayon/.cache/torch/hub/pytorch_vision_v0.10.0
/home/ayon/.conda/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ayon/.conda/envs/pytorch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# load the model with torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load('./../TL_weightsv2/resnet50.pt').to(device)
print(model)

/tmp/ipykernel_52876/2564688753.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./../TL_weightsv2/resnet50.pt').to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

model.fc = Identity()
# model.layer4 = Identity()

# add transform to data and resize to 224x224
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224)), torchvision.transforms.ToTensor()])

# load CIFAR10 dataset from torchvision
train_data = torchvision.datasets.OxfordIIITPet(root='./../../../Data', split='trainval', download=True, transform=transform)
test_data = torchvision.datasets.OxfordIIITPet(root='./../../../Data', split='test', download=True, transform=transform)

# add data to loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
# pass data through model and stack it as a tensor
train_features = torch.Tensor()
train_labels = torch.Tensor()
test_features = torch.Tensor()
test_labels = torch.Tensor()
for data, label in tqdm(train_loader):
    output = model(data.to(device)).cpu().detach()
    train_features = torch.cat((train_features, output))
    train_labels = torch.cat((train_labels, label))

for data, label in tqdm(test_loader):
    output = model(data.to(device)).cpu().detach()
    test_features = torch.cat((test_features, output))
    test_labels = torch.cat((test_labels, label))

 74%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                           | 43/58 [00:12<00:04,  3.39it/s]

In [ ]:
np.save('./../TL_featuresv2/pets37_train_features_resnet50_pretrained_imagenet.npy', train_features.numpy())
np.save('./../TL_featuresv2/pets37_train_labels_resnet50_pretrained_imagenet.npy', train_labels.numpy())

np.save('./../TL_featuresv2/pets37_test_features_resnet50_pretrained_imagenet.npy', test_features.numpy())
np.save('./../TL_featuresv2/pets37_test_labels_resnet50_pretrained_imagenet.npy', test_labels.numpy())